In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
from airflow.hooks import S3Hook
s3_conn = S3Hook().get_conn()
from skills_ml.datasets.job_postings import job_postings, job_postings_chain
import random
import json
from skills_ml.algorithms.corpus_creators.basic import Doc2VecGensimCorpusCreator, CorpusCreator
from collections import Counter
import numpy as np
from skills_ml.algorithms.sampling.jobs import JobSampler

[2017-11-03 15:07:36,262] {__init__.py:36} INFO - Using executor SequentialExecutor
[2017-11-03 15:07:37,459] {textcleaner.py:20} INFO - 'pattern' package not found; tag filters are not available for English


# Streaming Data to Create Corpus

## Simple Corpus

In [2]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator)

In [3]:
corpus = list(corpus)

[2017-11-03 15:07:37,922] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-03 15:07:38,017] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-03 15:07:38,019] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-03 15:07:38,171] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-03 15:07:38,793] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-03 15:07:39,032] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-03 15:07:39,246] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-03 15:07:39,424] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-03 15:07:39,619] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-03 15:07:39,826] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-03 15:07:40,005] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-03 15:07:45,278] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/NLX_b8384025-fa09-417d-ae10-96880fac86be
[2017-11-03 15:07:45,446] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-03 15:07:46,769] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-03 15:07:47,096] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-03 15:07:47,347] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-03 15:07:47,567] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-03 15:07:47,794] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-03 15:07:48,024] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-03 15:07:48,247] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-03 15:07:48,458] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-03 15:07:48,728] {

In [4]:
len(corpus)

10567

In [5]:
corpus[0]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'alternateName': 'Payroll Specialist',
 'baseSalary': {'@type': 'MonetaryAmount', 'maxValue': 0.0, 'minValue': 0.0},
 'datePosted': '2012-06-20',
 'description': 'Position Summary: The Payroll Specialist is responsible for processing all payroll information for accurate and timely payroll distribution. Responsibilities: Process store-level biweekly payroll accurately and timely Import store employee info and time clock data Research and resolve employee payroll issues Process involuntary wage withholding correspondence Respond promptly to all requests and communications received Review and process adjustments to hours/earnings Maintain employee direct deposit records Assist with various projects as assigned GREAT OPPORTUNITY!',
 'educationRequirements': 'Graduate Degree',
 'employmentType': 'Full-Time',
 'experienceRequirements': 'Qualifications: 1-3 years of general payroll experience Positive attitude combined with excellent 

## Creating Corpus with Criteria

#### One can define their own funciton and logic of filtering based on the common schema.

In [6]:
def major_group_filter_func(document):
    if document['onet_soc_code']:
        if document['onet_soc_code'][:2] in ['11', '13']:
            return document

def full_soc_code_filter_func(document):
    if document['onet_soc_code']:
        if document['onet_soc_code'] in ['11-9051.00', '17-3026.00']:
            return document

def wage_filter_func(document):
    if document['baseSalary']['minValue']:
        if float(document['baseSalary']['minValue']) >= 60000.0:
            return document

### Filtered by Major Groups

In [7]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=major_group_filter_func)

In [8]:
corpus = list(corpus)

[2017-11-03 15:07:54,402] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-03 15:07:54,502] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-03 15:07:54,503] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-03 15:07:54,654] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-03 15:07:55,157] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-03 15:07:55,433] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-03 15:07:55,631] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-03 15:07:55,831] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-03 15:07:56,060] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-03 15:07:56,291] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-03 15:07:56,494] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-03 15:07:57,324] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-03 15:08:00,015] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-03 15:08:00,878] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-03 15:08:01,344] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-03 15:08:01,680] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-03 15:08:02,053] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-03 15:08:02,353] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-03 15:08:02,612] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-03 15:08:02,847] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-03 15:08:03,079] {s3.py:83} INFO - 20171705 bytes transferred out of 20171705 total
[2017-11-03 15:08:03,488] {job_postings.py:36} INFO - Extracting job postings fro

In [9]:
corpus[0]

{'@context': 'http://schema.org',
 '@type': 'JobPosting',
 'alternateName': 'General Manager',
 'baseSalary': {'@type': 'MonetaryAmount', 'maxValue': 0.0, 'minValue': 0.0},
 'datePosted': '2012-09-17',
 'description': 'If you are a personable, outgoing and service-oriented professional who wants to manage a dynamic office, come join the Kool Smiles family! Kool Smiles was founded in 2002 and was built on the belief that every family has the right to quality dental care in a clean, safe and fun environment. Our first goal is to provide quality and compliant dental care to individuals in communities that typically get overlooked by other dentists. Our services aren’t just for kids- we know that children are more likely to have great dental habits if mom and dad do too, so we happily care for the entire family. Kool Smiles is looking for a customer-service-oriented leader to fill the role of Office Manager. As the Office Manager, you will manage an office staff of 15 to 30 employees and p

In [10]:
major_group = list(map(lambda c: c['onet_soc_code'][:2], corpus))

In [11]:
Counter(major_group)

Counter({'11': 283, '13': 102})

### Filtered by Full O*NET SOC Code

In [12]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=full_soc_code_filter_func)

In [13]:
corpus = list(corpus)

[2017-11-03 15:08:03,857] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-03 15:08:03,946] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-03 15:08:03,947] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-03 15:08:04,075] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-03 15:08:05,065] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-03 15:08:05,310] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-03 15:08:05,545] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-03 15:08:05,764] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-03 15:08:05,989] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-03 15:08:06,177] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-03 15:08:06,334] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-03 15:08:07,314] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-03 15:08:07,558] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-03 15:08:07,798] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-03 15:08:07,971] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-03 15:08:08,169] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-03 15:08:08,379] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-03 15:08:08,546] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-03 15:08:08,730] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-03 15:08:08,916] {s3.py:83} INFO - 20171705 bytes transferred out of 20171705 total
[2017-11-03 15:08:09,183] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/VA_c3b9dad2-ef6c-4439-a4e7-6dff0a55d362
[2017-11-03 15:08:09,

In [14]:
soc = list(map(lambda c: c['onet_soc_code'], corpus))

In [15]:
Counter(soc)

Counter({'11-9051.00': 79, '17-3026.00': 65})

### Filtered by Minimum Wage >= 60000

In [16]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=wage_filter_func)

In [17]:
corpus = list(corpus)

[2017-11-03 15:08:09,564] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-03 15:08:09,634] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-03 15:08:09,635] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-03 15:08:09,735] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-03 15:08:09,954] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-03 15:08:10,139] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-03 15:08:10,334] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-03 15:08:10,511] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-03 15:08:10,752] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-03 15:08:10,930] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-03 15:08:11,079] {s

In [18]:
min_wage = list(map(lambda c: c['baseSalary']['minValue'], corpus))

In [19]:
Counter(min_wage)

Counter({60000.0: 8,
         65000.0: 10,
         70000.0: 4,
         71110: 1,
         75000.0: 5,
         80000.0: 10,
         83221: 1,
         85000.0: 1,
         90000.0: 12,
         100000.0: 2,
         120000.0: 3,
         160000.0: 1})

# Sampling from Corpus

## Sampling from simple corpus

In [20]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator)

In [21]:
from skills_ml.algorithms.sampling.jobs import JobSampler
job_sampler = JobSampler(corpus, random_state=42)
corpus = job_sampler.sample(100)

[2017-11-03 15:08:14,068] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-03 15:08:14,133] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-03 15:08:14,134] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-03 15:08:14,224] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-03 15:08:14,610] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-03 15:08:14,827] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-03 15:08:14,998] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-03 15:08:15,172] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-03 15:08:15,351] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-03 15:08:15,505] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-03 15:08:15,665] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-03 15:08:21,113] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/NLX_b8384025-fa09-417d-ae10-96880fac86be
[2017-11-03 15:08:21,203] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-03 15:08:21,828] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-03 15:08:22,062] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-03 15:08:22,274] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-03 15:08:22,460] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-03 15:08:22,713] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-03 15:08:22,942] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-03 15:08:23,126] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-03 15:08:23,346] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-03 15:08:23,532] {

In [22]:
corpus[0]

({'@context': 'http://schema.org',
  '@type': 'JobPosting',
  'alternateName': 'Customer Service Representative',
  'baseSalary': {'@type': 'MonetaryAmount',
   'maxValue': 11.0,
   'minValue': 11.0},
  'datePosted': '2011-08-25',
  'description': 'We are currently looking for bilingual (Spanish- English) customer service representative. You need to be fluent in both languages, clerical experience, be customer service oriented and have a professional appearance. If you think you have great customer service skills, have a strong work ethic and want to be part of a great team please contact me',
  'educationRequirements': 'Not Specified',
  'employmentType': 'Full-Time',
  'experienceRequirements': 'MINIMUM QUALIFICATIONS: High school diploma and some college coursework or equivalent Capable of solving a variety of customer service problems and supporting a range of personalities and customer types Bi-lingual: Spanish Ability to communicate effectively with others, orally and written',
 

In [23]:
industry = list(map(lambda c: c[0]['industry'], corpus))

In [24]:
Counter(industry)

Counter({'': 59,
         'Advertising, Sales - Marketing, Hospitality': 1,
         'Biotechnology, Pharmaceutical, Food': 1,
         'Consulting, Sales - Marketing, Hospitality': 1,
         'Consumer Products, Other, Food': 1,
         'Consumer Products, Sales - Marketing, Food': 2,
         'Education - Teaching - Administration, Social Services, Food': 1,
         'Entertainment, Food, Hospitality': 1,
         'Entertainment, Retail, Hospitality': 1,
         'Hospitality, Restaurant, Food': 4,
         'Not for Profit - Charitable, Healthcare - Health Services, Food': 1,
         'Packaging, Manufacturing, Food': 1,
         'Public Relations, Not for Profit - Charitable, Food': 1,
         'Restaurant, Food, Hospitality': 7,
         'Restaurant, Full Service Restaurant, Hospitality': 1,
         'Restaurant, Managed Care, Hospitality': 1,
         'Restaurant, Retail, Hospitality': 6,
         'Retail, Restaurant, Food': 1,
         'Sales - Marketing, Consulting, Hospitalit

## Sampling from filtered corpus

### Reservoir Sampling

In [25]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=major_group_filter_func)

In [26]:
job_sampler = JobSampler(corpus, random_state=42)
corpus = job_sampler.sample(100)

[2017-11-03 15:08:28,971] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-03 15:08:29,037] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-03 15:08:29,039] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-03 15:08:29,124] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-03 15:08:29,878] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-03 15:08:30,191] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-03 15:08:30,417] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-03 15:08:30,631] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-03 15:08:30,806] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-03 15:08:30,998] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-03 15:08:31,174] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-03 15:08:31,894] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-03 15:08:32,158] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-03 15:08:32,454] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-03 15:08:32,651] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-03 15:08:32,887] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-03 15:08:33,059] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-03 15:08:33,239] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-03 15:08:33,431] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-03 15:08:33,641] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-03 15:08:33,820] {s3.py:83} INFO - 20171705 bytes transferred out of 20171705 total
[2017-11-03 15:08:34,261] {job_postings.py:36} INFO - Extracting job postings fro

In [27]:
onet_soc_code = list(map(lambda c: c[0]['onet_soc_code'][:2], corpus))

In [28]:
Counter(onet_soc_code)

Counter({'11': 78, '13': 22})

### Weighted Reservoir Sampling

In [29]:
job_postings_generator = job_postings_chain(s3_conn, ['2011Q2'], 'open-skills-private/test_corpus')
corpus = CorpusCreator(job_postings_generator, filter_func=major_group_filter_func)

In [30]:
job_sampler = JobSampler(corpus, major_group=True,weights={'11': 1, '13': 3.5})
sampled_corpus = job_sampler.sample(50)

[2017-11-03 15:08:34,665] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/
[2017-11-03 15:08:34,737] {s3.py:83} INFO - 0 bytes transferred out of 0 total
[2017-11-03 15:08:34,738] {job_postings.py:36} INFO - Extracting job postings from key test_corpus/2011Q2/CB_6874581a-5942-48fd-ba7e-2b1cdc38563f
[2017-11-03 15:08:34,806] {s3.py:83} INFO - 0 bytes transferred out of 18588132 total
[2017-11-03 15:08:35,462] {s3.py:83} INFO - 2072576 bytes transferred out of 18588132 total
[2017-11-03 15:08:35,694] {s3.py:83} INFO - 4145152 bytes transferred out of 18588132 total
[2017-11-03 15:08:35,906] {s3.py:83} INFO - 6217728 bytes transferred out of 18588132 total
[2017-11-03 15:08:36,179] {s3.py:83} INFO - 8290304 bytes transferred out of 18588132 total
[2017-11-03 15:08:36,354] {s3.py:83} INFO - 10362880 bytes transferred out of 18588132 total
[2017-11-03 15:08:36,548] {s3.py:83} INFO - 12435456 bytes transferred out of 18588132 total
[2017-11-03 15:08:36,745] {s

/Users/Eddie/Documents/DSSG/WDI/skills-ml/venv/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


[2017-11-03 15:08:37,568] {s3.py:83} INFO - 0 bytes transferred out of 20171705 total
[2017-11-03 15:08:37,853] {s3.py:83} INFO - 2244608 bytes transferred out of 20171705 total
[2017-11-03 15:08:38,043] {s3.py:83} INFO - 4489216 bytes transferred out of 20171705 total
[2017-11-03 15:08:38,230] {s3.py:83} INFO - 6733824 bytes transferred out of 20171705 total
[2017-11-03 15:08:38,408] {s3.py:83} INFO - 8978432 bytes transferred out of 20171705 total
[2017-11-03 15:08:38,586] {s3.py:83} INFO - 11223040 bytes transferred out of 20171705 total
[2017-11-03 15:08:38,780] {s3.py:83} INFO - 13467648 bytes transferred out of 20171705 total
[2017-11-03 15:08:38,943] {s3.py:83} INFO - 15712256 bytes transferred out of 20171705 total
[2017-11-03 15:08:39,113] {s3.py:83} INFO - 17956864 bytes transferred out of 20171705 total
[2017-11-03 15:08:39,296] {s3.py:83} INFO - 20171705 bytes transferred out of 20171705 total
[2017-11-03 15:08:39,744] {job_postings.py:36} INFO - Extracting job postings fro

In [31]:
major_group = list(map(lambda c: c[1][:2], sampled_corpus))

In [32]:
Counter(major_group)

Counter({'11': 23, '13': 27})

In [33]:
sampled_corpus[0]

({'@context': 'http://schema.org',
  '@type': 'JobPosting',
  'baseSalary': {'@type': 'MonetaryAmount', 'maxValue': '', 'minValue': ''},
  'datePosted': '2011-01-19',
  'description': 'Regional Staff Accountant Tracking Code 6466 Job Description Make your mark in Broadcasting and Digital Media. Sinclair Broadcast Group and Sinclair Digital Solutions are dedicated to making Sinclair a communications powerhouse! We are the largest and most diversified television broadcasting company in the country today. Sinclair owns and operates, programs or provides services to 162 stations located in 79 geographically diverse markets and our Digital group is focused on bringing the most engaging content to web, mobile and over-the-top broadcasting to audiences all over the country! Our success is the result of extraordinary employees and an exemplary management team who believe in a vision and are dedicated ensuring a great future for our employees. Whether you are an industry veteran or a just start